In [48]:
%run Predictions.ipynb

[*********************100%%**********************]  1 of 1 completed

2024-08-16 01:15:23.755742
2004-08-16 01:15:23.755742



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


           Date  ^GSPC_change  ^GSPC_lag        Close        Price
0    2004-08-16     14.539917        NaN  1079.339966  1064.800049
1    2004-08-17      2.369995  14.539917  1081.709961  1079.339966
2    2004-08-18     13.460083   2.369995  1095.170044  1081.709961
3    2004-08-19     -3.940063  13.460083  1091.229980  1095.170044
4    2004-08-20      7.119995  -3.940063  1098.349976  1091.229980
...         ...           ...        ...          ...          ...
5030 2024-08-09     37.219727  62.090332  5344.160156  5314.660156
5031 2024-08-12     25.100098  37.219727  5344.390137  5351.879883
5032 2024-08-13     65.379883  25.100098  5434.430176  5376.979980
5033 2024-08-14     58.770020  65.379883  5455.209961  5442.359863
5034 2024-08-15           NaN  58.770020  5530.959961  5501.129883

[5035 rows x 5 columns]
           Date  ^GSPC_change  ^GSPC_lag        Close        Price  \
0    2004-08-16     14.539917        NaN  1079.339966  1064.800049   
1    2004-08-17      2.369995  

### Prepare entire dataset for calculating profits and backtesting

In [49]:
scaler = StandardScaler()
X_scaled_temp = scaler.fit_transform(input_df)
X_scaled = pd.DataFrame(X_scaled_temp, columns = input_df.columns)

In [50]:
X_seq, y_seq = create_sequences(input_df, output_df)
print(X_seq.shape)
print(y_seq.shape)

torch.Size([4248, 10, 9])
torch.Size([4248, 1])


### Final Model Predictions on entire Dataset

In [51]:
dataset = TensorDataset(X_seq, y_seq)

final_loader = DataLoader(dataset, shuffle=False)

In [75]:
model.eval()

predictions = []

with torch.no_grad():
    test_mse = []
    test_mape = []
    for X_batch, y_batch in final_loader:
        test_predictions = model(X_batch)
        mse = criterion(test_predictions, y_batch)
        mape = get_mape(y_batch, test_predictions)
        predictions.append(test_predictions)
        test_mse.append(mse.item())
        test_mape.append(mape)
    
    average_mse_loss = sum(test_mse) / len(test_mse)
    average_mape_loss = sum(test_mape) / len(test_mape)
    print(f'Average MSE: {average_mse_loss:.2f}, Average MAPE: {average_mape_loss:.2f}')

predictions_tensors = torch.cat(predictions, dim=0)
predictions_list = predictions_tensors.flatten().tolist()

Average MSE: 1186.80, Average MAPE: 225122.16
